In [1]:
# training xgboost

In [2]:
import pandas as pd
import numpy as np

In [5]:

df = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv")
df.head()


,sample_id,catalog_content,image_link,price,item_name,bullet_points,product_description,value,unit,quantity_standard,unit_standard,quantity_numeric,quantity_count,price_per_unit
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",NaN,NaN,72.00,Fl,NaN,Fl_Oz,2129.292000,NaN,0.002297
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Salerno Cookies, The Original Butter Cookies, ...",Original Butter Cookies: Classic butter cookie...,NaN,32.00,Ounce,907.184000,Ounce,907.184000,NaN,0.014462
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Bear Creek Hearty Soup Bowl, Creamy Chicken wi...",Loaded with hearty long grain wild rice and ve...,NaN,11.40,Ounce,323.184300,Ounce,323.184300,NaN,0.006096
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Judee’s Blue Cheese Powder 11.25 oz - Gluten-F...,"Add to your favorite appetizers, dips & spread...",Judees Powdered Blue Cheese cheddar cheese pow...,11.25,Ounce,318.931875,Ounce,318.931875,NaN,0.095130
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...","kedem Sherry Cooking Wine, 12.7 Ounce - 12 per...",NaN,12.00,Count,12.000000,Count,12.000000,12.0,NaN


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re

# Combine text features
df["text_all"] = (
    df["item_name"].fillna("") + " " +
    df["bullet_points"].fillna("") + " " +
    df["product_description"].fillna("")
)

# Basic text cleaning
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 ]', ' ', text)  # remove special chars
    text = re.sub(r'\s+', ' ', text)             # normalize spaces
    return text.lower().strip()

df["text_all"] = df["text_all"].apply(clean_text)

In [7]:
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2)  # include unigrams and bigrams
)

X_tfidf = tfidf.fit_transform(df["text_all"])
print("TF-IDF shape:", X_tfidf.shape)


TF-IDF shape: (75000, 5000)


In [8]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

# Select numeric features
numeric_features = ["quantity_numeric", "price_per_unit", "value"]
X_num = df[numeric_features].fillna(0)

# One-hot encode unit_standard
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
X_cat = ohe.fit_transform(df[["unit_standard"]])

# Combine all (TF-IDF + numeric + categorical)
from scipy.sparse import csr_matrix
X_final = hstack([csr_matrix(X_num.values), X_cat, X_tfidf])

y = df["price"]

print("Final feature shape:", X_final.shape)

Final feature shape: (75000, 5016)


In [ ]:
import numpy as np

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0  # avoid division by zero
    return np.mean(diff) * 100


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_final, df["price"], test_size=0.2, random_state=42
)


In [ ]:
from xgboost import XGBRegressor
import numpy as np

# --- Step 1: Optionally cap outliers ---
lower_cap = df['price'].quantile(0.01)
upper_cap = df['price'].quantile(0.99)
df['price_capped'] = df['price'].clip(lower=lower_cap, upper=upper_cap)

# --- Step 2: Log-transform prices to handle skew/outliers ---
y_log = np.log1p(df['price_capped'])

# --- Step 3: Train/test split ---
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_final, y_log, test_size=0.2, random_state=42
)

# --- Step 4: Define XGBoost model ---
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=500,        # reduced to save memory
    learning_rate=0.05,
    max_depth=6,             # reduced to save memory
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42
)

# --- Step 5: Fit model directly on sparse matrices ---
xgb_model.fit(
    X_train, 
    y_train,
    eval_set=[(X_val, y_val)],
    
    verbose=True
)

# --- Step 6: Predict on full dataset ---
y_pred_log = xgb_model.predict(X_final)
y_pred = np.expm1(y_pred_log)        # revert log transform
y_pred[y_pred > 200] = 200           # cap prices at 200 if needed




[0]	validation_0-rmse:0.91045
[1]	validation_0-rmse:0.88174
[2]	validation_0-rmse:0.85479
[3]	validation_0-rmse:0.82922
[4]	validation_0-rmse:0.80649
[5]	validation_0-rmse:0.78648
[6]	validation_0-rmse:0.76558
[7]	validation_0-rmse:0.74620
[8]	validation_0-rmse:0.73154
[9]	validation_0-rmse:0.71355
[10]	validation_0-rmse:0.69742
[11]	validation_0-rmse:0.69536
[12]	validation_0-rmse:0.68977
[13]	validation_0-rmse:0.67550
[14]	validation_0-rmse:0.66204
[15]	validation_0-rmse:0.64859
[16]	validation_0-rmse:0.63832
[17]	validation_0-rmse:0.62875
[18]	validation_0-rmse:0.61846
[19]	validation_0-rmse:0.60785
[20]	validation_0-rmse:0.59882
[21]	validation_0-rmse:0.58971
[22]	validation_0-rmse:0.58168
[23]	validation_0-rmse:0.57444
[24]	validation_0-rmse:0.56688
[25]	validation_0-rmse:0.56009
[26]	validation_0-rmse:0.55468
[27]	validation_0-rmse:0.54923
[28]	validation_0-rmse:0.54388
[29]	validation_0-rmse:0.54362
[30]	validation_0-rmse:0.53964
[31]	validation_0-rmse:0.53495
[32]	validation_0-

In [41]:
import numpy as np

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0  # handle division by zero
    return np.mean(diff) * 100

# SMAPE on full dataset
smape_full = smape(df['price'].values, df['predicted_price'].values)
print(f"Full dataset SMAPE: {smape_full:.2f}%")

Full dataset SMAPE: 17.46%


In [38]:
# Assume df has a column 'sample_id' which identifies each row
# If not, create it as a simple index
if 'sample_id' not in df.columns:
    df['sample_id'] = np.arange(1, len(df) + 1)

# Create submission DataFrame
submission_df = df[['sample_id']].copy()
submission_df['price'] = df['predicted_price']

# Save to CSV
submission_df.to_csv('output2.csv', index=False)

print("Submission CSV 'output2.csv' created successfully!")


Submission CSV 'output2.csv' created successfully!


In [39]:
import joblib

# Save the model
joblib.dump(xgb_model, "xgb_model.pkl")
print("✅ Model saved as xgb_model.pkl")

# Load it later
xgb_model_loaded = joblib.load("xgb_model.pkl")


✅ Model saved as xgb_model.pkl


In [40]:
import joblib

# Save TF-IDF vectorizer
joblib.dump(tfidf, "tfidf_vectorizer.pkl")
print("✅ TF-IDF vectorizer saved as tfidf_vectorizer.pkl")

# Save OneHotEncoder
joblib.dump(ohe, "onehot_encoder.pkl")
print("✅ OneHotEncoder saved as onehot_encoder.pkl")


✅ TF-IDF vectorizer saved as tfidf_vectorizer.pkl
✅ OneHotEncoder saved as onehot_encoder.pkl
